In [161]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [121]:
import time
from tqdm import tqdm_notebook

In [142]:
def get_soup(url):
    req = requests.get(url)
    if req.status_code == 200:
        html = req.text
        html = re.sub('<!-- -->', '', html)
        soup = BeautifulSoup(html, 'lxml')
        return soup
    return None

Поля:

* Услуги и курсы
* Тип библиотеки
* Сайт
* Электронный каталог

### Тренировка на одной из страниц

In [143]:
soup = get_soup('http://www.bibliogorod.ru/libraries/1')

In [134]:
clean(soup.find('div', class_="name").text)

'Библиотека им. Н. А. Некрасова'

In [149]:
def clean(line):
    line = re.sub('^ ?-? | $|\n|\r|\xa0', '', line)
    return line


def parse_libr(soup):
    
    name = clean(soup.find('div', class_="name").text)
    cls_names = ["border", "information_item"]
    d = {}
    
    for index, item in enumerate(cls_names):
        
        sp = soup.find_all('div', class_=item)
        
        for line in sp:
            
            inf = list(line.children)
           
            if len(inf) > 1:
                cat = clean(inf[0].text)
            
                if index == 0:
                    d[cat] = clean(inf[1].text)
    
                else:
                    f = line.find_all('div', class_='list')
                    d[cat] = [clean(l.text) for l in f]
   
    return name, d

### Парсим все

In [150]:
link = 'http://www.bibliogorod.ru/libraries/'
all_libr_info = {}

t_par = tqdm_notebook([i for i in range(1, 460)], desc='link', leave=True)

for i in t_par:
    
    try:
        soup = get_soup(link + str(i))
        name, info = parse_libr(soup)
        all_libr_info[name] = info
    except:
        print(link + str(i))

    time.sleep(2)


http://www.bibliogorod.ru/libraries/38
http://www.bibliogorod.ru/libraries/111
http://www.bibliogorod.ru/libraries/132
http://www.bibliogorod.ru/libraries/186
http://www.bibliogorod.ru/libraries/274
http://www.bibliogorod.ru/libraries/289
http://www.bibliogorod.ru/libraries/290
http://www.bibliogorod.ru/libraries/291
http://www.bibliogorod.ru/libraries/292
http://www.bibliogorod.ru/libraries/306
http://www.bibliogorod.ru/libraries/307
http://www.bibliogorod.ru/libraries/399
http://www.bibliogorod.ru/libraries/425
http://www.bibliogorod.ru/libraries/459



In [159]:
# 'Электронный каталог'
# 'Тип библиотеки'

d = {'Название':[], 'Электронный каталог':[], 'Тип библиотеки':[]}

for libr in all_libr_info:
    
    ctlg = all_libr_info[libr].get('Электронный каталог')
    tp = all_libr_info[libr].get('Тип библиотеки')
    
    d['Название'].append(libr)
    d['Электронный каталог'].append(ctlg)
    d['Тип библиотеки'].append(tp)
    

In [163]:
data = pd.DataFrame(d)
data.head()

,Название,Тип библиотеки,Электронный каталог
0,Библиотека им. Н. А. Некрасова,универсальная,opac.nekrasovka.ru/vmk/
1,Пункт книговыдачи Библиотеки им. Н. А. Некрасо...,универсальная,opac.nekrasovka.ru/vmk/
2,Пункт книговыдачи Библиотеки им. Н. А. Некрасо...,универсальная,opac.nekrasovka.ru/vmk/
3,Центральная детская библиотека №78 им. И.А. Кр...,детская,62.117.87.150/MarcWeb/
4,Центральная детская библиотека №66,детская,opac.nekrasovka.ru/opacg2/?bibl


In [165]:
data.to_csv('bibliogorod.csv')